# Implementing simulated preprocessing phase 
This phase produces a global MAC key $\alpha$ and secret shares $\alpha_i$ and random pairs $(\llbracket r \rrbracket, r_{open})$ 

In [1]:
#Import necessary libraries
from dataclasses import dataclass
from typing import List, Tuple
import random

In [5]:
# Large prime field for SPDZ arithmetic
P = 2**61 - 1

def mod(x: int) -> int:
    return x % P

@dataclass
class ShareRepr:
    delta: int
    shares: List[int]
    mac_shares: List[int]

class PreprocessingDealer:
    def __init__(self, n: int, seed: int = None):
        if seed is not None:
            random.seed(seed)
        self.n = n
        
        # Generate global MAC key alpha
        self.alpha = mod(random.randrange(P))

        # Secret-share alpha: alpha = sum(alpha_i)
        self.alpha_shares = [mod(random.randrange(P)) for _ in range(n - 1)]
        self.alpha_shares.append(mod(self.alpha - sum(self.alpha_shares)))

        # Storage for random pairs and triples
        self.pairs: List[Tuple[ShareRepr, int]] = []
        self.triples: List[Tuple[ShareRepr, ShareRepr, ShareRepr]] = []

    # ----------------------------------------------------
    # Create a secret-shared representation with MAC
    # ----------------------------------------------------
    def make_secret_shared(self, value: int, delta: int = 0):
        shares = [mod(random.randrange(P)) for _ in range(self.n - 1)]
        shares.append(mod(value - sum(shares)))

        mac_shares = [
            mod(self.alpha_shares[i] * mod(value + delta))
            for i in range(self.n)
        ]
        return shares, mac_shares

    # ----------------------------------------------------
    # Generate random pair 〈r〉 and open r
    # ----------------------------------------------------
    def gen_pair(self) -> Tuple[ShareRepr, int]:
        r = mod(random.randrange(P))
        shares, macs = self.make_secret_shared(r, delta=0)
        r_repr = ShareRepr(delta=0, shares=shares, mac_shares=macs)
        self.pairs.append((r_repr, r))
        return r_repr, r

    # ----------------------------------------------------
    # Generate Beaver triple 〈a〉,〈b〉,〈c〉 with c=a*b
    # ----------------------------------------------------
    def gen_triple(self, a=None, b=None):
        if a is None:
            a = mod(random.randrange(P))
        if b is None:
            b = mod(random.randrange(P))
        c = mod(a * b)

        sa, ma = self.make_secret_shared(a)
        sb, mb = self.make_secret_shared(b)
        sc, mc = self.make_secret_shared(c)

        a_repr = ShareRepr(0, sa, ma)
        b_repr = ShareRepr(0, sb, mb)
        c_repr = ShareRepr(0, sc, mc)

        self.triples.append((a_repr, b_repr, c_repr))
        return a_repr, b_repr, c_repr

    # ----------------------------------------------------
    # Bulk generation
    # ----------------------------------------------------
    def bulk_generate(self, num_pairs=0, num_triples=0):
        for _ in range(num_pairs):
            self.gen_pair()
        for _ in range(num_triples):
            self.gen_triple()

    # ----------------------------------------------------
    # Triple verification: sacrifice method
    # (filters out bad triples)
    # ----------------------------------------------------
    def check_triples_sacrifice(self, triples=None):
        if triples is None:
            triples = self.triples

        valid = []
        i = 0

        while i + 1 < len(triples):
            (aR, bR, cR) = triples[i]
            (fR, gR, hR) = triples[i + 1]

            # reconstruct shares
            a = mod(sum(aR.shares))
            b = mod(sum(bR.shares))
            c = mod(sum(cR.shares))
            f = mod(sum(fR.shares))
            g = mod(sum(gR.shares))
            h = mod(sum(hR.shares))

            # random t
            t = mod(random.randrange(P))

            rho = mod(t * a - f)
            sigma = mod(b - g)

            Q = mod(t * c - h - sigma * f - rho * g - mod(sigma * rho))

            if Q == 0:
                # triple is valid
                valid.append((aR, bR, cR))
            # else discard (a,b,c)

            i += 2

        self.triples = valid
        return valid

    # ----------------------------------------------------
    # Inject a BAD triple for testing
    # ----------------------------------------------------
    def inject_bad_triple(self):
        a = mod(random.randrange(P))
        b = mod(random.randrange(P))
        c = mod(random.randrange(P))  # WRONG on purpose

        sa, ma = self.make_secret_shared(a)
        sb, mb = self.make_secret_shared(b)
        sc, mc = self.make_secret_shared(c)

        a_repr = ShareRepr(0, sa, ma)
        b_repr = ShareRepr(0, sb, mb)
        c_repr = ShareRepr(0, sc, mc)

        self.triples.append((a_repr, b_repr, c_repr))
        return a_repr, b_repr, c_repr


# ----------------------------------------------------
# Example Usage
# ----------------------------------------------------
if __name__ == "__main__":
    dealer = PreprocessingDealer(n=3, seed=42)

    dealer.bulk_generate(num_pairs=5, num_triples=5)
    dealer.inject_bad_triple()

    print("Generated triples (including bad ones):", len(dealer.triples))

    valid = dealer.check_triples_sacrifice()
    print("Valid triples after sacrifice:", len(valid))

    # Show a sample pair
    if dealer.pairs:
        r_repr, r_open = dealer.pairs[0]
        print("Example pair r =", r_open)
        print("Shares =", r_repr.shares)
        print("MAC shares =", r_repr.mac_shares)


Generated triples (including bad ones): 6
Valid triples after sacrifice: 2
Example pair r = 321752549611034334
Shares = [236337776990707881, 1707863537360842093, 683394244473178311]
MAC shares = [1119557825249448184, 923923729542464201, 1724303791925356512]
